In [53]:
import memray
import numpy as np
import os
from IPython.display import HTML, display
from numpy           import array
from scipy           import linalg
from scipy.linalg    import lu, lu_factor
from timeit          import repeat

# Einsendeaufgabe 1: Numerische Genauigkeit und Gleichungssysteme (100 Punkte)
In dieser Aufgabe sollen Sie ein wenig mehr Erfahrung mit NumPy und numerischen Methoden gewinnen.  
Zur Erinnerung empfehle ich an dieser Stelle, die Definition der [IEEE-Fließkommazahlen](https://de.wikipedia.org/wiki/IEEE_754) zu wiederholen.  

## Addition von Zahlen (20 Punkte) 

Gegeben sei ein Array *array*, dass 100 mal die Zahl $10^{-16}$ enthält und einmal (als ersten Eintrag) die Zahl $1$. 

In [54]:

array = np.concatenate(([1], np.full(100, 1e-16)))
print(array) 

[1.e+00 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16]


**Aufgabe:** Addieren Sie alle Werte in `array`:
- mit der Funktion `np.sum()`
- mit einer Schleife, die von **vorne nach hinten** über `array` iteriert
- mit einer Schleife, die von **hinten nach vorne** über `array` iteriert

_Points:_ 6

<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">
    
## Addition von Zahlen im Array: Unterschiedliche Berechnungsmethoden

In dieser Aufgabe geht es um die Addition von Zahlen, die in einem Array gespeichert sind. Das Array besteht aus 101 Einträgen, wobei der erste Eintrag die Zahl 1 enthält und die restlichen 100 Einträge jeweils den Wert $ 1 \times 10^{-16} $ haben. Die Aufgabe besteht darin, die Summe der Werte in diesem Array auf verschiedene Arten zu berechnen: einmal mit der `np.sum()`-Funktion und zweimal unter Verwendung von Schleifen, die das Array jeweils von vorne nach hinten bzw. von hinten nach vorne durchlaufen.

<divv>

In [55]:
sum_forward = 0.0
for x in array:
    sum_forward += x
print(f"Summe mit Schleife: {sum_forward=}")

sum_reverse = 0.0
for x in reversed(array):
    sum_reverse += x
print(f"Summe mit umgekehrter Schleife: {sum_reverse=}")

sum_numpy = np.sum(array)
print(f"Summe mit np.sum: {sum_numpy=}")

Summe mit Schleife: sum_forward=np.float64(1.0)
Summe mit umgekehrter Schleife: sum_reverse=np.float64(1.00000000000001)
Summe mit np.sum: sum_numpy=np.float64(1.0000000000000084)


<!-- BEGIN QUESTION -->

**Aufgabe:** Erklären Sie die Ergebnisse. Wie werden die Zahlen 1 und 1e-17 im Computer dargestellt? (ausführliche Erläuterung erforderlich!)  

_Points:_ 12

<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">
    

## Numerische Genauigkeit von verschiedenen Additionsverfahren

### Unterschiede in der Summierung

Bei der Berechnung der Summe mit unterschiedlichen Verfahren ergeben sich leicht unterschiedliche Ergebnisse:

```python
sum_numpy   = np.float64(1.0000000000000084)
sum_forward = np.float64(1.0)
sum_reverse = np.float64(1.00000000000001)
```
Diese Unterschiede entstehen durch **Rundungsfehler** in der **Gleitkommadarstellung**, die auf der begrenzten Anzahl an Bits basiert, mit der reale Zahlen intern im Computer dargestellt werden.

---

## Gleitkommazahlen im IEEE 754 Standard

Python und NumPy verwenden standardmäßig `float64` (IEEE 754 Double Precision), das wie folgt aufgebaut ist:

- **1 Bit** für das Vorzeichen  
- **11 Bits** für den Exponenten  
- **52 Bits** für die Mantisse  

Insgesamt also **64 Bit**. Die reale Zahl wird durch folgende Formel dargestellt:

```
Zahl = (-1)^s · 1.m · 2^(e - 1023)
```

Dabei ist:

- `s` das Vorzeichenbit  
- `m` die Mantisse (ohne führende 1)  
- `e` der gespeicherte Exponent mit **Bias** = 1023  

---

## Maschinengenauigkeit

Die kleinste Zahl, die zu `1.0` addiert werden kann, sodass sich das Ergebnis **numerisch unterscheidet**, nennt man **Maschinengenauigkeit** oder **machine epsilon** $\varepsilon_{\text{mach}}$:

```python
np.finfo(float).eps
# Ergebnis: 2.220446049250313e-16
```

Zahlen kleiner als `eps` „verpuffen“ bei der Addition zu größeren Zahlen wie `1.0`, da sie außerhalb der darstellbaren Genauigkeit der Mantisse liegen. Wenn zwei Gleitkommazahlen stark unterschiedlich groß sind, kann die kleinere Zahl beim Addieren „verschluckt“ werden – das nennt man Verlust der Signifikanz:

```python
print(1e-16 < np.finfo(float).eps)  # True
print(1.0 + 1e-16 == 1.0)           # True
```

---

## Warum liefert `sum_forward` ein anderes Ergebnis als `sum_reverse`?

Angenommen:

```python
array = np.array([1.0] + [1e-16] * 100)
```

- `sum_forward` addiert zuerst `1.0` und dann viele kleine `1e-16`. Diese einzelnen Summanden sind kleiner als `eps` und **gehen verloren**.
- `sum_reverse` addiert zuerst die kleinen `1e-16` zu einem größeren Zwischenwert (z. B. `1e-14`), der dann zu `1.0` addiert wird und sich **merklich auswirkt**.
- `sum_numpy` verwendet oft optimierte oder paarweise Additionen (`pairwise summation`), was ebenfalls zu einem etwas besseren Ergebnis führen kann.

Gleitkommazahlen sind **nicht assoziativ**, d. h.:

```
(a + b) + c ≠ a + (b + c)
```

Das bedeutet, dass die Reihenfolge der Addition das Ergebnis beeinflussen kann, besonders wenn sehr kleine Zahlen in der Summe enthalten sind.

---

## Darstellung der Zahl `1.0` im IEEE 754-Standard

#### Mathematische Form:

$
1.0 = 1.0 \cdot 2^0 \\
\Rightarrow \text{Echter Exponent} = 0 \\
\Rightarrow \text{Gespeicherter Exponent} = 0 + \text{Bias} = 1023 = 01111111111_2
$

#### Mantisse:
- Die Zahl `1.0` entspricht im Binärsystem: $ 1.000\ldots0_2 $
- Die **führende 1 vor dem Komma** wird im IEEE 754-Format **nicht gespeichert** (implizit angenommen)
- Alle 52 Bits der Mantisse (also der Nachkommastellen) sind daher **0**

#### IEEE 754-Darstellung (64 Bit – `double precision`):

|      s      |   e (11 Bit)   |                    m (52 Bit)                          |
|-------------|----------------|--------------------------------------------------------|
|     `0`     |  `01111111111` | `0000000000000000000000000000000000000000000000000000` |

#### Zusammenfassung:
- **Vorzeichenbit (`s`)**: `0` → Zahl ist positiv  
- **Exponent (`e`)**: 1023 → gespeichert als `01111111111`  
- **Mantisse (`m`)**: nur Nullen (weil keine Nachkommastellen vorhanden sind)  
- Daraus ergibt sich der dargestellte Wert:
  
$
(-1)^0 \cdot 1.0 \cdot 2^{0} = 1.0
$

---

## Darstellung der Zahl `1e-17` im IEEE 754-Standard (Double Precision)

### Mathematische Zerlegung von `1e-17`

### Ziel: Darstellung in der Form

Wir suchen $m$ und $e$, sodass:

$1e-17 = 1.m \cdot 2^{e - 1023}$

### Schritt 1: Umwandlung in Zweierpotenz

Zunächst schreiben wir `1e-17` in etwa als Zweierpotenz:

$1e-17 = 10^{-17} \approx (2^{3.321928094887354...})^{-17}\approx 2^{-3.321928094887354... \cdot 17} \approx 2^{-56.47}$

Das bedeutet:

$1e-17 \in (2^{-57}, 2^{-56})$

### Schritt 2: Rundung auf nächstliegende Zweierpotenz

Da `1e-17` bzw. $2^{-56.47}$ näher an $2^{-57}$ liegt, weil:

$\lvert 2^{-56.47}-2^{-57} \rvert = \lvert  \frac{1}{2^{56.47}} - \frac{1}{2^{57}} \rvert < \lvert  \frac{1}{2^{56.47}} - \frac{1}{2^{56}} \rvert= \lvert 2^{-56.47} - 2^{-56} \rvert$

gilt:

$1e-17 \approx 1.m \cdot 2^{-57}$

### Schritt 3: Exponent berechnen

Der effektive Exponent ist:

$e_{\text{effektiv}} = -57$

Im IEEE 754 Double Precision wird der **Exponent mit Bias 1023** gespeichert:

$e_{\text{gespeichert}} = -57 + 1023 = 966$

In Binär:

$966_{10} = 01111000110_2$

### Schritt 4: Berechnung der Mantisse

Es gilt $1e-17 \approx 1.m \cdot 2^{-57}$ und somit $1.m = \frac{1e-17}{2^{-57}} = 1.4411518807585587...$

Wir nehmen die Nachkommastellen von `1.44115188...` und wandeln sie in Binär um. 

Wir schreiben `1e-17` in binärer Normalform:

$1e-17 = 1.0111000011101111010101000110010001101101010010010110\dots_2 \cdot 2^{-57}$

Die Mantisse sind **die 52 Bits nach dem Komma** (die führende $1.$ wird nicht gespeichert):

### Schritt 5: Zusammensetzung der IEEE-754-Bitfolge

| Feld        | Wert                                                         |
|-------------|--------------------------------------------------------------|
| Vorzeichen  | `0` (da positiv)                                             |
| Exponent    | `01111000110` (`11` Bit für `966`)                           |
| Mantisse    | `0111000011101111010101000110010001101101010010010110`       |


### Gesamter 64-Bit-IEEE-754-Code (Binär)

Die IEEE-754-Bitfolge sieht wie folgt aus (in 64 Bit):
|      s      |   e (11 Bit)   |                    m (52 Bit)                          |
|-------------|----------------|--------------------------------------------------------|
|     `0`     |  `01111000110` | `0111000011101111010101000110010001101101010010010110` |

---
### Fazit:

- Die Zahl `1.0` kann exakt im Format IEE 754 `float64` dargestellt werden.  
- Die Zahl `1e-17` ist **zu klein** und hat eine **unendliche Binärdarstellung**, was zu einer _Rundung_ führen muss. Eine präzise Darstellung innerhalb der 52 Bits der Mantisse ist nicht möglich

</div>
-

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Welches Verfahren verwendet `np.sum()` für die Addition?

_Points:_ 2

<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">

## Kaskadensummation

Paarweise Summation (auch Kaskadensummation genannt) ist ein numerisch besseres Verfahren, bei dem die Werte in mehreren Schritten summiert werden. Dies bedeutet, dass Werte mit ähnlichen Größen zuerst zusammengefasst werden. Dadurch wird der Fehler reduziert, der entstehen kann, wenn immer kleinere Zahlen zu einem sehr großen Wert hinzugefügt werden (wie es bei der sequentiellen Summation der Fall wäre).

### Verfahren von `np.sum()`

NumPy verwendet für die Addition von Arrays ein Verfahren namens **Kaskadensummation** oder **paarweise Summation**. Dieses Verfahren reduziert Rundungsfehler im Vergleich zur sequentiellen Summation und führt in vielen Fällen zu einer höheren Genauigkeit. `np.sum()` kann interne Optimierungen verwenden, wie z. B. parallelisierte Summation, abhängig von der Größe des Arrays und der Hardware. In modernen Implementierungen von NumPy wird oft die `np.add.reduce()` Funktion verwendet, um die Summation durchzuführen, die im Hintergrund mit dem `ufunc`-Mechanismus arbeitet. Diese `reduce`-Methode summiert die Werte sequenziell, aber auch sie optimiert den Prozess, indem sie die Reihenfolge der Summanden variiert oder bei Bedarf parallelisiert.


Mehr Informationen zu dieser Technik gibt es auf [Wikipedia](https://en.wikipedia.org/wiki/Pairwise_summation) und in der [NumPy-Dokumentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

</div>

<!-- END QUESTION -->

## Grenzwerte von Funktionen (20 Punkte)

Geben sei die Funktion $f(x)=\frac{e^x-1}{x}$

Es gilt: $lim_{x \to 0}f(x) = 1$

**Aufgabe:** Schreiben Sie eine Python Funktion `f(x)`, mit deren Hilfe Sie $f(x)$ für $x= 10^{-1}, 10^{-2}, \cdots , 10^{-15}$ mit NumPy berechnen und geben Sie das Ergebnis aus.

**Aufgabe:** Berechnen Sie nun die Funktion für  $x= 10^{-16}, 10^{-17}, \cdots , 10^{-20}$

_Points:_ 8

<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">

<h2>Berechnung der Funktion f(x) = (exp(x) - 1) / x </h2>

In dieser Aufgabe soll die Funktion f(x) für x = 10<sup>-1</sup>, 10<sup>-2</sup>, ..., 10<sup>-15</sup> berechnet werden. Danach soll die Funktion für noch kleinere Werte von x, nämlich x = 10<sup>-16</sup>, 10<sup>-17</sup>, ..., 10<sup>-20</sup>, berechnet werden. 

Der Grenzwert der Funktion für x → 0 ist 1. Dabei sollen mögliche numerische Probleme bei sehr kleinen x beobachtet werden.

</div>

In [56]:
# Definition der Funktion f(x) = (e^x - 1)/x
def f(x):
        return (np.exp(x) - 1) / x

# Bereich 1: x = 10^(-1) bis 10^(-15)
# Erzeuge die Werte x = 10^(-1), 10^(-2), ..., 10^(-15)
input_numbers = np.array([10**(-i) for i in range(1, 16)])
result_limits = f(input_numbers)

#Ausgabe
for x_val, fx in zip(input_numbers, result_limits):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-01, f(x) = 1.0517091807564771
x = 1e-02, f(x) = 1.005016708416795
x = 1e-03, f(x) = 1.0005001667083846
x = 1e-04, f(x) = 1.000050001667141
x = 1e-05, f(x) = 1.000005000006965
x = 1e-06, f(x) = 1.0000004999621837
x = 1e-07, f(x) = 1.0000000494336803
x = 1e-08, f(x) = 0.999999993922529
x = 1e-09, f(x) = 1.000000082740371
x = 1e-10, f(x) = 1.000000082740371
x = 1e-11, f(x) = 1.000000082740371
x = 1e-12, f(x) = 1.000088900582341
x = 1e-13, f(x) = 0.9992007221626409
x = 1e-14, f(x) = 0.9992007221626409
x = 1e-15, f(x) = 1.1102230246251565


In [57]:
# Bereich 2: x = 10^(-16) bis 10^(-20)
input_numbers_2 = np.array([10**(-i) for i in range(16, 21)])
result_limits_2 = f(input_numbers_2)

#Ausgabe
for x_val, fx in zip(input_numbers_2, result_limits_2):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-16, f(x) = 0.0
x = 1e-17, f(x) = 0.0
x = 1e-18, f(x) = 0.0
x = 1e-19, f(x) = 0.0
x = 1e-20, f(x) = 0.0


<!-- BEGIN QUESTION -->

**Aufgabe:** Was fällt auf? Welche Erklärung haben Sie für das Ergebnis? (Ausführliche Erklärung erforderlich) 

_Points:_ 12

<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">
    
<h2> Erklärung des Verhaltens der Funktion bei kleinen x-Werten </h2>

Für Werte von x zwischen `1e-01` bis `1e-14` liefert die Funktion erwartungsgemäß Ergebnisse nahe bei **1**. Ab etwa 1e-15 treten jedoch drastische Änderungen auf, die durch Rundungsfehler bei der Berechnung von `e^x - 1` bedingt sind. Besonders bei sehr kleinen x-Werten wird der Unterschied zwischen `e^x` und `1` auf dem Computer nicht mehr erkennbar, was zu fehlerhaften Ergebnissen führt. Eine stabilere Berechnung kann durch die Verwendung der Funktion `np.expm1(x)` von NumPy erreicht werden.


Von `1e-01` bis `1e-14` ergibt die Funktion erwartungsgemäß Werte, die sehr nahe bei **1** liegen.  Doch ab etwa `1e-15` ändert sich das Verhalten drastisch:

`x = 1e-15, f(x) = 1.1102230246251565`

`x = 1e-16, f(x) = 0.0`

`x = 1e-17, f(x) = 0.0`  
...

**Bei genauerer Betrachtung** sieht man, dass bereits ab `1e-08` praktisch keine Annäherung mehr an den Soll-Grenzwert stattfindet. Die Berechnung "divergiert numerisch".

| Phänomen                    | Beschreibung                                                                |
|-----------------------------|------------------------------------------------------------------------------|
| Katastrophale [Auslöschung]( https://de.m.wikipedia.org/wiki/Ausl%C3%B6schung_%28numerische_Mathematik%29 )   | Verlust signifikanter Ziffern durch Subtraktion ähnlicher Zahlen             |
| Numerische Instabilität     | Instabiles Verhalten bei kleinen Änderungen der Eingabe                      |
| Rundungsfehlerakkumulation  | Anhäufung von Maschinenfehlern durch viele Rechenoperationen                 |
| (Numerische) Divergenz      | Umgangssprachlich, wenn das Ergebnis sich vom Sollwert entfernt              |

### Ergebnisbeobachtung

Ab etwa $x \approx 10^{-16}$ treten numerische Rundungsfehler sichtbar auf, abhängig von der Maschinengenauigkeit (`float64`).

<h3> Erklärung: Rundungsfehler </h3>

Für sehr kleine `x` gilt näherungsweise:  
`exp(x) ≈ 1 + x`

Wenn `x` so klein wird, dass `e^x` und `1` auf dem Computer **nicht mehr unterscheidbar** sind, wird der Ausdruck `e^x - 1` **numerisch ungenau** (_null_), da beide Terme sehr nah beieinander liegen → Verlust signifikanter Stellen.

> Beispiel:  
> `x = 1e-17` → `exp(x) ≈ 1.00000000000000001`  
> Die Differenz `e^x - 1` ist kleiner als die **Maschinengenauigkeit** `ε` (Epsilon).  
> Deshalb wird `e^x - 1` vom Computer als **0.0** berechnet, da ein 64-Bit-Gleitkommawert (float64) **nicht mehr ausreichend genau** ist.

<h3> Lösung: Stabilere Berechnung mit NumPy </h3>

NumPy bietet mit `np.expm1(x)` eine spezielle Funktion an, die den Ausdruck `exp(x) - 1` **numerisch stabiler** berechnet – insbesondere für sehr kleine `x`.

</div>

In [58]:
def f_stabil(x):
    return np.expm1(x) / x

input_numbers_3 = np.array([10**(-i) for i in range(10, 20)])
result_limits_3 = f_stabil(input_numbers_3)

#Ausgabe
for x_val, fx in zip(input_numbers_3, result_limits_3):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-10, f(x) = 1.00000000005
x = 1e-11, f(x) = 1.000000000005
x = 1e-12, f(x) = 1.0000000000005
x = 1e-13, f(x) = 1.00000000000005
x = 1e-14, f(x) = 1.000000000000005
x = 1e-15, f(x) = 1.0000000000000007
x = 1e-16, f(x) = 1.0
x = 1e-17, f(x) = 1.0
x = 1e-18, f(x) = 1.0
x = 1e-19, f(x) = 1.0


<div style="color: brown; background-color: #f5f5dc; padding: 5px; border-radius: 5px;" title="Ahmed">

## Detaillierte Erklärung der Gleitkommadarstellung und Rundungsfehler

Die Präzision einer Gleitpunktzahl hängt von der Mantisse ab. Diese bildet neben dem Exponenten, der Basis und dem Vorzeichen die Gleitpunktzahl.

Solange man nicht explizit davon abweicht, wird `float64` in Python und NumPy verwendet für Gleitpunktzahlen. Eine `float32`-Mantisse besteht aus 23 Bit und eine `float64`-Mantisse aus 52 Bit (bei 64 Bit bleiben noch 11 Bit für den Exponenten und 1 Bit für das Vorzeichen). Bei Werten, die mehr Bits zur Darstellung benötigen als die Mantisse zur Verfügung stellt, kommt es zu Rundungsfehlern, wie in den folgenden Beispielen erklärt:

## Fazit zur Gleitkommadarstellung und Rundungsfehlern

Die Präzision von Gleitkommazahlen ist durch die Anzahl der Bits in der Mantisse begrenzt, was zu Rundungsfehlern führt, wenn Zahlen mit mehr erforderlichen Bits dargestellt werden müssen. Besonders bei sehr kleinen Werten kann dies zu Verlusten an Genauigkeit führen, die sich durch den Einsatz stabiler Berechnungsmethoden wie z. B. `np.expm1(x)` verhindern lassen.

</div>

In [59]:
def mindest_mantissenbits(x, y):
    # Berechnet die Anzahl der benötigten Mantissenbits für eine korrekte Berechnung
    # float128 um exaktere Praezision zu ermoeglichen
    return np.abs(np.log2(np.float128(x)) - np.log2(np.float128(y)))

# Beispiel zur Darstellbarkeit float32
# Mantisse = 1 wenn dargestellte Zahl Potenz von 2, also darstellbar
x = 1
a = np.float128(2**24-1)
print(x + a, np.float32(x+a), "darstellbar?:", mindest_mantissenbits(x,a) < 23 + 1)
# Ergebnis: (16777216.0, 16777216.0, 'darstellbar?:', True)

# Beispiel zur Darstellbarkeit float32
# Dargestellte Zahl muss hier mit Mantisse von Mindestlaenge 24 dargestellt werden, nicht moeglich bei float32
b = np.float128(2**24)
print(x + b, np.float64(x+b), "darstellbar?:", mindest_mantissenbits(x,b) < 23 + 1)
# Ergebnis: (16777217.0, 16777216.0, 'darstellbar?:', False)

# Diese Beispiele verdeutlichen, wie die Genauigkeit und die Darstellung von Zahlen durch die verwendete Gleitkommadarstellung beeinflusst werden.

# Weitere Beispiele zur Rundungsfehlerdarstellung
# In den folgenden Beispielen wird die Auswirkung von kleinen Zahlen in Gleitkommadarstellungen demonstriert:

# Beispiel 1 + 1e-16, vorkommend bei Summierung in Schleife von vorne
c = np.float128(1e-16)
print(x + c, np.float64(x + c), "darstellbar?:", mindest_mantissenbits(x, c) < 52 + 1)
# Ergebnis: (1.0000000000000001, 1.0, 'darstellbar?:', False)

# Beispiel 1 + 1e-16 * 100, vorkommend bei Summierung in Schleife von hinten
d = np.float128(1e-16 * 100)
print(x + d, np.float64(x + d), "darstellbar?:", mindest_mantissenbits(x, d) < 52 + 1)
# Ergebnis: (1.00000000000001, 1.00000000000001, 'darstellbar?:', True)


16777216.0 16777216.0 darstellbar?: True
16777217.0 16777217.0 darstellbar?: False
1.0000000000000001 1.0 darstellbar?: False
1.00000000000001 1.00000000000001 darstellbar?: True


<!-- END QUESTION -->

## Gauß-Verfahren und LU-Zerlegung (60 Punkte) 
Obwohl man, wie wir sehen werden, lineare Gleichungssysteme einfach mit SciPy lösen kann, wollen wir in dieser Aufgabe  einmal selbst eine sogenannte LU-Zerlegung bzw. das Gauß-Verfahren implementieren. Außerdem werden wir diesen Code auch später noch verwenden, wenn wir uns mit Performance-Optimierung und Parallelisierung beschäftigen.

Das Gauß-Verfahren ist ein wichtiges Verfahren zum Lösen von linearen Gleichungssystemen, wir gehen hier schrittweise mit der Implementierung vor.

**Aufgabe:** Implementieren Sie die Funktion `b_solve` zur Lösung einer Dreiecksgleichung der Form:

$\left[ {\begin{array}{cccc} 
a_{11} & a_{12} & \cdots & a_{1n}\\ 
0 & a_{22} & \cdots & a_{2n}\\ 
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \cdots & a_{nn}\\ 
\end{array} } \right] \left[{\begin{array}{c}  
x_1\\ 
x_2\\ 
\vdots\\ 
x_n\\ 
\end{array}}\right] =  
\left[{\begin{array}{c}  
b_1\\ 
b_2\\ 
\vdots\\ 
b_n\\ 
\end{array}}\right]$ 

Hier beginnt man mit der Löung der letzten Zeile, denn es gilt:

$x_n = b_n/a_{nn} $
und dann

$x_{n-1} =  (b_{n-1}-a_{(n-1),n}*x_{n})/a_{(n-1),(n-1)}$

$x_{n-2} =  (b_{n-2}-(a_{(n-2),n}*x_{n}+a_{(n-2),(n-1)}*x_{n-1})/a_{(n-2),(n-2)}$


*Hinweis*: Achten Sie darauf, dass ihr Code nicht nur korrekt, sondern auch effizient ist. 

_Points:_ 12

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

# Lösung linearer Gleichungssysteme mit dem Gauß-Verfahren und LU-Zerlegung

In dieser Aufgabe geht es um die Implementierung eines grundlegenden numerischen Verfahrens zur Lösung von linearen Gleichungssystemen der Form:

$A \cdot x = b$

wobei:
- $A$ eine Matrix (n × n) ist,
- $x$ der gesuchte Lösungsvektor,
- $b$ der rechte Seitenvektor ist.

Anstelle der Verwendung fertiger Bibliotheksfunktionen (wie `scipy.linalg.solve`) wird hier das klassische **Gaußsche Eliminationsverfahren** mit anschließender **Rückwärtseinsetzung** selbst implementiert. Ziel ist es, ein tieferes Verständnis für die numerischen Methoden zu erlangen und eine solide Basis für spätere Optimierungen und Parallelisierungen zu schaffen.

---

## Hintergrund

Das Gauß-Verfahren ist eine Standardmethode in der numerischen Mathematik, um lineare Gleichungssysteme zu lösen. Es besteht aus zwei Hauptschritten:

1. **Vorwärtselimination (Dreiecksform)**  
   Die Matrix $A$ wird durch gezielte Zeilenoperationen in eine obere Dreiecksmatrix umgewandelt. Dies bedeutet, dass alle Einträge unterhalb der Hauptdiagonalen zu Null gemacht werden. Um numerische Stabilität sicherzustellen, wird **Pivotisierung** eingesetzt – das größte Element einer Spalte wird nach oben getauscht.

2. **Rückwärtseinsetzung (Backsubstitution)**  
   Sobald die Matrix in Dreiecksform ist, kann die Lösung des Systems zeilenweise von unten nach oben bestimmt werden. Beginnend mit der letzten Gleichung wird jede Unbekannte explizit ausgerechnet.

---

## Umsetzung

Zwei zentrale Funktionen werden implementiert:

- **`make_triangular(A, b)`**  
  Bringt die Matrix $A$ durch Vorwärtselimination und Pivotisierung in eine obere Dreiecksform, wobei notwendige Operationen auch auf den Vektor $b$ angewandt werden.

- **`b_solve(A, b, upper_triangular=True)`**  
  Löst das Gleichungssystem durch Rückwärtseinsetzung. Falls `upper_triangular=False` übergeben wird, ruft die Funktion intern `make_triangular` auf, um die Matrix zuerst in Dreiecksform zu bringen.

Besonderes Augenmerk wird auf:
- **Numerische Stabilität** (durch Pivotisierung),
- **Effizienz** (durch vektorisiertes Rechnen mit `numpy`),
- **Korrekte Fehlerbehandlung**

---

## Zusammenfassung

Durch diese Implementierung wird ein vollständiger, effizienter und robuster Ansatz zum Lösen von linearen Gleichungssystemen bereitgestellt, 
der später als Grundlage für Optimierungs- und Parallelisierungsstrategien dienen kann. 

Die Implementierung legt besonderen Wert auf numerische Stabilität durch Pivotisierung der Matrix während der Vorwärtselimination. 

Dadurch wird sichergestellt, dass numerische Fehler minimiert und stabile Lösungen auch bei ungünstig skalierten Systemen erreicht werden.B
 und Parallelisierungsstrategien dienen kann.

</div>

In [60]:
def b_solve(A, b, upper_triangular=True):
    """Löst ein lineares Gleichungssystem A * x = b"""

    x = np.zeros_like(b, dtype=float)  # Vektor für die Lösung, initialisiert mit Nullen
    A = A.astype(float)  # Sicherstellen, dass mit floats gerechnet wird
    b = b.astype(float)
    
    #Matrix in Dreiecksform umwandeln wenn nötig
    if not upper_triangular:
        make_triangular(A, b)        
    
    # Rückwärtseinsetzung
    for i in range(len(b) - 1, -1, -1):  
        # Berechnung der Summe der Produkte der bekannten x-Werte
        summation = np.dot(A[i, i + 1:], x[i + 1:]) # hier wird die Vektorisierung, die numpy zur Verfügung stellt, benutzt, um die Effizienz zu erhöhen
        x[i] = (b[i] - summation) / A[i, i]  # Berechnung des aktuellen x_i
        #print(f"x[{i}] = (b[{i}] - summation) / A[i, i] = ({b[i]} - {np.dot(A[i, i + 1:], x[i + 1:])}) / {A[i,i]} = {x[i]}")
    return x

def make_triangular(A, b):
    """bringt Matrix A in Dreiecksform unter Berücksichtigung von Vektor b"""
    
    # Vorwärtselimination (Dreiecksform erzeugen)
    for i in range(len(b)):
        # Pivotisierung: größtes Element in der Spalte nach unten holen (numerische Stabilität)
        max_row = i + np.argmax(abs(A[i:, i]))
        if A[max_row, i] == 0:
            raise ValueError("Das Gleichungssystem hat keine eindeutige Lösung")
        if max_row != i:
            A[[i, max_row]] = A[[max_row, i]]
            b[[i, max_row]] = b[[max_row, i]]

        # Elimination
        for j in range(i + 1, len(b)):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]

# Beispiel
A = np.array([[2, 3, 5, 1],
              [0, 4, 6, 2],
              [0, 0, 7, 3],
              [0, 0, 0, 8]], dtype=float)

b = np.array([9, 8, 7, 8], dtype=float)

x = b_solve(A, b)

# Testausgabe
print("Lösung x:", x)
print("Prüfung A @ x:", np.dot(A, x))  # Sollte gleich b sein

Lösung x: [1.60714286 0.64285714 0.57142857 1.        ]
Prüfung A @ x: [9. 8. 7. 8.]


**Aufgabe:** Implementieren Sie die LU-Zerlegung einer Matrix mit Pivotisierung.  
Die Funktion `lu_decomposition` soll die Matrizen L und U berechnen.  
Zu Beginn wird der Funktion eine Matrix A übergeben.  


Implementieren Sie die Funktion "in-place", d.h. die Ergebnisse für L und U werden direkt in A geschrieben.  
Dabei wird die ursprüngliche Matrix überschrieben.  
Die Funktion gibt den Pivot-Vektor zurück, der die folgende Form haben sollte:

$ 
P=\left[{\begin{array}{ccc} 
0 & 1& 0\\ 
0 & 0& 1\\ 
1 & 0& 0\\ 
\end{array} } \right] \Rightarrow \left[{\begin{array}{c}  
1\\ 
2\\ 
0\\ 
\end{array}}\right] 
$ 

D.h. es wird nur der Index des Wertes in jeder Zeile gespeichert, der nicht Null ist.

Wenn man dies macht, kann man später für die Lösung einfach den Vektor `bc = b[P]` verwenden, um auch $b$ richtig zu pivotisieren.  

Hinweise: 
- Die Pivot-Zeile lässt sich am einfachsten mit `np.argmax` finden. Die Zeile mit dem höchsten Betrag können Sie mit `np.abs` finden.

- Tauschen zweier Elemente in einem NumPy Array: `v[[a, b]] = v[[b, a]]`

- Versuchen Sie, die interne Vektorisierung von NumPy im Eliminationsschritt zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 15

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

# LU-Zerlegung mit Pivotisierung

In dieser Aufgabe geht es um die Implementierung der **LU-Zerlegung** einer Matrix unter Berücksichtigung von **Pivotisierung**.  
Die LU-Zerlegung ist ein zentrales Verfahren der numerischen Mathematik und linearen Algebra, das eine Matrix $A$ in die Produktform zweier spezieller Matrizen zerlegt:

$A = L \cdot U$

wobei:
- $L$ eine untere Dreiecksmatrix mit Einsen auf der Hauptdiagonalen ist (unit lower triangular matrix),
- $U$ eine obere Dreiecksmatrix ist.

Zusätzlich wird bei dieser Zerlegung eine **Pivotisierung** durchgeführt, um numerische Stabilität sicherzustellen. Die Pivotisierung wird durch eine Permutationsmatrix $P$ beschrieben, sodass im Allgemeinen gilt:

$P \cdot A = L \cdot U$

---

## Hintergrund

Warum Pivotisierung?  
Ohne Pivotisierung könnten bei der Zerlegung kleine Pivotelemente entstehen, die zu großen numerischen Fehlern führen.  
Deshalb wird stets das Element mit dem größten Betrag in der aktuellen Spalte als Pivot gewählt, und die entsprechenden Zeilen werden getauscht.

Im Algorithmus wird die Pivotinformation in Form eines **Pivot-Vektors** $P$ gespeichert.  
Dieser Vektor gibt an, welche Zeilen bei der Pivotisierung wie vertauscht wurden. Dadurch kann später einfach der Vektor $b$ angepasst werden, indem man `b[P]` verwendet.

Beispiel eines Pivot-Vektors:
$
P = \left[ \begin{array}{c} 1 \\ 2 \\ 0 \end{array} \right]
$
Dieser beschreibt eine Permutationsmatrix der Form:

$
\left[ \begin{array}{ccc} 
0 & 1 & 0\\ 
0 & 0 & 1\\ 
1 & 0 & 0
\end{array} \right]
$

---

## Umsetzung

Die Implementierung erfolgt **in-place**:  
- Die ursprüngliche Matrix $A$ wird während der Berechnungen überschrieben.
- Die Einträge der unteren Dreiecksmatrix $L$ (ohne Einsen auf der Diagonalen) und der oberen Dreiecksmatrix $U$ werden direkt in $A$ gespeichert.

Die Schritte im Algorithmus sind:

1. **Pivotisierung**:  
   In jeder Spalte wird der größte Betrag gesucht, die Zeilen werden entsprechend getauscht.

2. **Berechnung von L**:  
   Die Einträge unterhalb der Hauptdiagonale werden berechnet, indem die aktuelle Spalte normiert wird.

3. **Berechnung von U**:  
   Durch Subtraktion eines äußeren Produkts wird die obere rechte Ecke der Matrix angepasst.

---

## Hinweise zur Umsetzung

- Der Pivotindex wird mit `np.argmax(np.abs(A[k:n, k])) + k` effizient bestimmt.
- Zeilentausch in NumPy-Arrays erfolgt mit `v[[a, b]] = v[[b, a]]`.
- Zur Effizienzsteigerung wird beim Eliminationsschritt **Vektorisierung** mit Slicing-Notation genutzt.
- Bei einer singulären Matrix (wenn das Pivotelement Null ist) wird eine Exception ausgelöst.

---

## Zusammenfassung

Durch diese Implementierung entsteht eine schnelle, speichereffiziente und numerisch stabile Methode zur LU-Zerlegung, die sowohl für spätere direkte Lösungenvon Gleichungssystemen als auch für Performance-Analysen geeignet ist.


</div>


In [61]:
def lu_decomposition(A):
    """L und U Matrix werden innerhalb von A Matrix berechnet und der Pivot-Vektor wird ausgegeben"""
    
    n = A.shape[0]
    P = np.arange(n)

    for k in range(n):
        # Pivotisierung: Index der Zeile mit größtem Betrag in Spalte k finden
        pivot_index = np.argmax(np.abs(A[k:n, k])) + k
        if A[pivot_index, k] == 0:
            raise ValueError("Matrix ist singulaer.")

        # Zeilen tauschen in A
        if pivot_index != k:
            A[[k, pivot_index], :] = A[[pivot_index, k], :]
            P[[k, pivot_index]] = P[[pivot_index, k]]

        # Berechnung L unterhalb der Diagonale
        A[k+1:n, k] /= A[k, k]

        # Berechnung U obere Dreiecksmatrix inklusive Diagonale
        A[k+1:n, k+1:n] -= np.outer(A[k+1:n, k], A[k, k+1:n])

    return P #Pivot-Vektor

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion `custom_solve`, welche mit Hilfe von `lu_decomposition` ein lineares Gleichungssystem der Form $A*x=b$ löst.  

Hinweis: Versuchen Sie, die interne Vektorisierung von NumPy zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 12

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

# Lösen eines linearen Gleichungssystems mit LU-Zerlegung

In dieser Aufgabe implementieren wir eine Funktion `custom_solve`, die ein lineares Gleichungssystem der Form

$A \cdot x = b$

löst.  
Dabei nutzen wir unsere eigene, zuvor implementierte Funktion `lu_decomposition`, die $A$ in eine untere Dreiecksmatrix $L$ und eine obere Dreiecksmatrix $U$ zerlegt.

---

## Hintergrund

Das Lösen eines Gleichungssystems über die LU-Zerlegung läuft in mehreren Schritten ab:

1. **LU-Zerlegung mit Pivotisierung**:  
   Die Matrix $A$ wird in $L$ und $U$ zerlegt, sodass gilt:

   $P \cdot A = L \cdot U$

   wobei $P$ eine Permutationsmatrix ist, die die Pivotisierungen beschreibt.

2. **Anpassung des rechten Seitenvektors $b$**:  
   Um die Pivotisierung zu berücksichtigen, wird $b$ entsprechend permutiert:

   $b' = b[P]$

3. **Zweischrittiges Lösen:**
   - Zunächst wird das Gleichungssystem

     $L \cdot c = b'$

     gelöst (Vorwärtseinsetzung).
   
   - Anschließend wird das Gleichungssystem

     $U \cdot x = c$

     gelöst (Rückwärtseinsetzung).

Dadurch erhalten wir die Lösung $x$ des ursprünglichen Gleichungssystems.

---

## Umsetzung im Detail

Die Funktion `custom_solve` folgt genau diesen Schritten:

1. **LU-Zerlegung**  
   Die Funktion `lu_decomposition(A)` wird aufgerufen. Dabei wird die Matrix $A$ in-place in ihre $L$- und $U$-Komponenten zerlegt und der Pivot-Vektor $P$ wird zurückgegeben.

2. **Extrahieren von $L$ und $U$**  
   - $L$ wird aus $A$ extrahiert, indem alle Einträge unterhalb der Hauptdiagonalen übernommen werden, zusammen mit Einsen auf der Diagonalen.
   - $U$ wird als obere Dreiecksmatrix inklusive Hauptdiagonale extrahiert.

3. **Vorwärtseinsetzung**  
   Das Gleichungssystem

   $L \cdot c = b[P]$

   wird gelöst.  
   Da $L$ keine obere Dreiecksmatrix ist, wird beim Aufruf von `b_solve` der Parameter `upper_triangular=False` gesetzt.

4. **Rückwärtseinsetzung**  
   Anschließend wird das Gleichungssystem

   $U \cdot x = c$

   gelöst, wobei $U$ eine obere Dreiecksmatrix ist. Hier wird `b_solve` mit Standardparametern aufgerufen.

5. **Rückgabe der Lösung**  
   Die Funktion gibt die berechnete Lösung $x$ zurück.

---

## Hinweise

- Die Funktion arbeitet mit **in-place Modifikationen**: $A$ wird während der LU-Zerlegung überschrieben.
- Die **interne Vektorisierung von NumPy** wird konsequent genutzt (z.B. Slicing-Syntax und Funktionen wie `np.tril` und `np.triu`), was den Code effizient und lesbar macht.
- Diese Methode ist insbesondere für große lineare Gleichungssysteme numerisch stabil und performant.

---

## Zusammenfassung

Die Funktion `custom_solve` bietet eine vollständige, saubere und effiziente Lösung eines linearen Gleichungssystems durch eigene Implementierungen der LU-Zerlegung und der Lösung von Dreieckssystemen.  
Damit wird eine der zentrale Techniken der numerischen linearen Algebra eigenständig nachvollzogen und umgesetzt.

</div> 

In [62]:
def custom_solve(A, b):
    """Löst ein lineares Gleichungssystem A * x = b und gib Vektor x aus"""
    
    #Pivot-Vektor und A (welches LU-Zerlegung enthaelt) werden berechnet
    P = lu_decomposition(A)
    
    #L und U werden aus A isoliert
    n = A.shape[0]
    L = np.tril(A, k=-1) + np.eye(n)  # untere Dreiecksmatrix mit Einsen auf der Diagonale
    U = np.triu(A)                   # obere Dreiecksmatrix inklusive Diagonale
    
    #Lc = b[P] wird geloest fuer c
    #hier wird upper_triangular=False verwendet um eine allgemeine Matrix zu loesen
    c = b_solve(L, b[P], upper_triangular=False)
    
    #Ux = c wird geloest fuer x
    x = b_solve(U, c)
    
    return x

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Testfunktion, die überprüft, ob die Lösung ihrer Implementierung des Gauß-Verfahrens richtig ist.  
Testen Sie Gleichungssysteme der Größe $16 \times 16$, $32\times 32$ und $64 \times 64$.

Sie können zum Vergleich die SciPy Funktion `linalg.solve` verwenden:

_Points:_ 6

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">


# Überprüfung der Lösung mit einer Testfunktion

In dieser Aufgabe sollen wir eine Testfunktion implementieren, die überprüft, ob die Lösung des Gauß-Verfahrens korrekt ist. Dies erreichen wir, indem wir das Ergebnis des eigenen Implementierungsansatzes mit dem Ergebnis der `linalg.solve`-Funktion aus dem SciPy-Paket vergleichen.

---

## Hintergrund

Das Gauß-Verfahren ist ein weit verbreitetes numerisches Verfahren zur Lösung von linearen Gleichungssystemen der Form

$A \cdot x = b$

In der Aufgabe haben wir bereits eine Funktion `custom_solve` implementiert, die dieses Gleichungssystem unter Verwendung der LU-Zerlegung löst. Jetzt möchten wir sicherstellen, dass unsere Implementierung korrekt funktioniert. Zu diesem Zweck führen wir eine Reihe von Tests durch, die die Ergebnisse mit der bekannten und bewährten SciPy-Funktion `linalg.solve` vergleichen.

---

## Vorgehensweise

Die Testfunktion `test_custom_solve` geht wie folgt vor:

1. **Erzeugen der Zufallsmatrix $A$ und des Vektors $b$**:  
   Für jedes Testdurchlauf wird eine Zufallsmatrix $A$ und ein Zufallsvektor $b$ der jeweiligen Dimension $n$ erzeugt. Dabei wird ein Seed für den Zufallsgenerator festgelegt, um die Ergebnisse reproduzierbar zu machen.

2. **Vergleich der Ergebnisse**:  
   Die Funktion `linalg.solve` aus SciPy wird verwendet, um das Gleichungssystem zu lösen und ein Referenzergebnis zu erhalten. Anschließend wird das Gleichungssystem auch mit der eigenen Implementierung `custom_solve` gelöst.

3. **Überprüfung der Genauigkeit**:  
   Es wird überprüft, ob die Ergebnisse der beiden Methoden (SciPy und eigene Implementierung) nahezu gleich sind. Dies geschieht unter Verwendung der Funktion `np.allclose`, die überprüft, ob die Ergebnisse innerhalb einer bestimmten Toleranz (hier: `1e-6`) übereinstimmen. Dies berücksichtigt mögliche numerische Ungenauigkeiten bei Fließkommaoperationen.

4. **Ausgabe**:  
   Wird der Test bestanden, gibt die Funktion eine Bestätigung aus, dass der Test für die jeweilige Dimension erfolgreich war.

---

## Hinweise

1. **Numerische Toleranz**:  
   Da numerische Berechnungen mit Fließkommazahlen in Computern immer mit kleinen Ungenauigkeiten verbunden sind, ist es wichtig, die Ergebnisse mit einer Toleranz zu vergleichen. Diese wird durch den Parameter `atol=1e-6` in der Funktion `np.allclose` festgelegt. Dieser Parameter sorgt dafür, dass geringe Unterschiede, die durch die begrenzte Präzision von Fließkommazahlen entstehen, ignoriert werden, ohne die Richtigkeit der Berechnungen zu beeinträchtigen.

2. **Dimensionen der Matrizen**:  
   Die Testfunktion prüft das Verhalten der Implementierung mit Matrizen unterschiedlicher Größen. Dies ist besonders wichtig, um sicherzustellen, dass die Implementierung auch für größere Matrizen effizient und korrekt arbeitet. In diesem Fall werden Matrizen der Größen $16 \times 16$, $32 \times 32$ und $64 \times 64$ getestet. Dadurch wird überprüft, ob die Funktion auch für größere Dimensionen ordnungsgemäß funktioniert.

---

## Zusammenfassung

Die Funktion `test_custom_solve` vergleicht die Ergebnisse der eigenen Implementierung des Gauß-Verfahrens mit der Lösung von SciPy, um die Korrektheit der Implementierung zu überprüfen. Die Tests werden mit Matrizen unterschiedlicher Größen durchgeführt, und die Toleranz für numerische Fehler wird berücksichtigt. Damit haben wir eine verlässliche Methode, die Funktionsfähigkeit und Genauigkeit unserer Implementierung sicherzustellen.

</div>

In [63]:
def test_custom_solve(n):    
    """Vergleicht Ergebnisse von custom_solve mit linalg.solve mittels assert-Bedingung"""
    
    rng = np.random.default_rng(seed=123)
    A = rng.random((n, n))         #Erzeugt eine Zufallsmatrix der Größe n x n
    b = rng.random(n)              #Erzeugt einen Zufallsvektor der Länge n
    linalg_x = linalg.solve(A, b)  #Löst das Gleichungssystem mit der SciPy-Funktion 
    custom_x = custom_solve(A, b)  #Löst das gleiche System mit der eigenen Implementierung
    
    #sicherstellen, dass Ergebnisse gleich sind, float-Unpraezision ignorieren
    assert np.allclose(linalg_x, custom_x, atol=1e-6), f"Ergebnisse weichen zu stark ab bei Dimension {n}"
    
    print("Test bestanden fuer Dimension", n)
    
test_custom_solve(16)
test_custom_solve(32)
test_custom_solve(64)

Test bestanden fuer Dimension 16
Test bestanden fuer Dimension 32
Test bestanden fuer Dimension 64


<!-- END QUESTION -->

**Aufgabe:** Wir wollen nun weiter die *fertigen* SciPy Funktionen betrachten. Mit den Funktionen `lu` und `lu_factor` aus dem Packet `scipy.linalg `  können Sie eine LU-Dekomposition durchführen [[1]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html#scipy.linalg.lu_factor) und [[2]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html).  
Beide Funktionen haben unter anderem den Parameter `overwrite_a`, der `False` oder `True` sein kann. 

Evaluieren Sie die Performance (Laufzeit) der LU-Zerlegung einer 4096x4096 Matrix, wenn Sie jeweils die Funktionen `lu_factor` und `lu` mit `overwrite_a = True` oder `overwrite_a = False` verwenden.

In der Liste `minimums` sollen die minimalen Laufzeiten in folgender Reihenfolge stehen:
1. `lu_factor` mit `overwrite_a = False`
2. `lu_factor` mit `overwrite_a = True`
3. `lu` mit `overwrite_a = False`
4. `lu` mit `overwrite_a = True`

Hinweis: Da die Matrix A bei der Verwendung von `overwrite_a = True` verändert wird, kann es zu einer Vereinfachung der Berechnung kommen, wenn die Funktion mehrfach nacheinander ausgeführt wird. Aus diesem Grund muss die `timeit` Funktion [repeat](https://docs.python.org/3/library/timeit.html#timeit.repeat) mit `repeat = 2` und `number = 1`  verwendet werden. Die Matrix wird im `setup` Parameter erstellt (aus Zufallswerten mit dem vorgegebenen "random number generator").

_Points:_ 6

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

## Performance-Evaluierung der LU-Zerlegung

In dieser Aufgabe soll die Performance (Laufzeit) der LU-Zerlegung einer $4096 \times 4096$ Matrix unter Verwendung der SciPy-Funktionen `lu` und `lu_factor` aus dem Modul `scipy.linalg` untersucht werden. Diese beiden Funktionen sind in der Lage, eine Matrix in ihre LU-Zerlegung zu zerlegen, wobei $L$ eine untere Dreiecksmatrix und $U$ eine obere Dreiecksmatrix ist.

Beide Funktionen haben den Parameter `overwrite_a`, der steuert, ob die Eingabematrix `A` während der Berechnung verändert werden darf. Der Parameter kann entweder auf `True` oder auf `False` gesetzt werden, was unterschiedliche Auswirkungen auf die Laufzeit und den Speicherbedarf haben kann.

### Ziel der Aufgabe

Das Ziel dieser Aufgabe ist es, die Performance der beiden Funktionen zu vergleichen und die minimalen Laufzeiten unter verschiedenen Bedingungen zu ermitteln. Dabei werden die folgenden Szenarien getestet:

1. **`lu_factor` mit `overwrite_a=False`**
2. **`lu_factor` mit `overwrite_a=True`**
3. **`lu` mit `overwrite_a=False`**
4. **`lu` mit `overwrite_a=True`**

Die minimalen Laufzeiten für diese vier Kombinationen sollen in einer Liste gespeichert werden, die wie folgt aussieht:
minimums = [ 1. lu_factor(A, overwrite_a=False), 2. lu_factor(A, overwrite_a=True), 3. lu(A, overwrite_a=False), 4. lu(A, overwrite_a=True) ]


### Vorgehensweise

1. **Matrix-Erstellung**:  
   Zunächst wird eine zufällige Matrix $A$ der Größe $4096 \times 4096$ erzeugt. Dies geschieht unter Verwendung des Random Number Generators (RNG) von NumPy.

2. **Laufzeitmessung**:  
   Zur Messung der Laufzeit wird die Python-Funktion `timeit.repeat` verwendet. Diese Funktion ermöglicht es, den Code mehrmals auszuführen (mit `repeat=2`), um eine genauere Messung der Ausführungszeit zu erhalten. Dabei wird der `setup`-Parameter verwendet, um die Matrix vor jeder Messung zu initialisieren. Die `number=1` stellt sicher, dass jede Messung nur einmal durchgeführt wird.

3. **Vergleich der Laufzeiten**:  
   Nach der Durchführung der Experimente werden die minimalen Laufzeiten für jede der vier getesteten Varianten ermittelt und in einer Liste `minimums` gespeichert.

### Lösung

Die Lösung verwendet die `timeit`-Funktion, um die Laufzeiten für die vier Kombinationen zu messen.

### Interpretation der Ergebnisse

- **`lu_factor` mit `overwrite_a=False`** hat die kürzeste Laufzeit, was darauf hindeutet, dass das Nicht-Überschreiben der Eingabematrix für diese Funktion die effizienteste Methode darstellt.
- **`lu_factor` mit `overwrite_a=True`** ist etwas langsamer, was möglicherweise darauf hindeutet, dass das Überschreiben der Matrix zusätzliche Rechenoperationen erfordert.
- **`lu` mit `overwrite_a=False`** ist langsamer als `lu_factor`, was darauf hinweist, dass die zusätzliche Funktionalität von `lu` (die die Matrix gleichzeitig in $L$ und $U$ zerlegt) zu einem größeren Rechenaufwand führt.
- **`lu` mit `overwrite_a=True`** ist etwas langsamer als `lu` mit `overwrite_a=False`, was darauf hindeutet, dass das Überschreiben der Matrix auch hier zu einem zusätzlichen Aufwand führt.

### Fazit

Die Ergebnisse bestätigen, dass `lu_factor` mit `overwrite_a=False` die effizienteste Methode für die LU-Zerlegung darstellt, während das Überschreiben der Matrix in der Regel einen leichten Performanceverlust zur Folge hat.

---

## Hinweise

- **Überprüfung der Performance**: Die Verwendung von `timeit.repeat` stellt sicher, dass die Laufzeiten zuverlässig und präzise gemessen werden, indem mehrere Wiederholungen durchgeführt werden.
- **Randomisierte Matrix**: Die Matrix wird bei jedem Lauf zufällig erzeugt, was bedeutet, dass die Tests unter realistischen Bedingungen durchgeführt werden, da die Eingabedaten für jedes Experiment unterschiedlich sind.
- **Überschreiben der Matrix**: Das Setzen von `overwrite_a=True` kann in einigen Fällen zu einer leicht verbesserten Performance führen, da weniger Speicher für die Eingabematrix erforderlich ist.

---

## Zusammenfassung

In dieser Aufgabe wurde die Performance der LU-Zerlegung für eine $4096 \times 4096$ Matrix unter Verwendung der SciPy-Funktionen `lu_factor` und `lu` untersucht. Es wurde getestet, ob das Überschreiben der Eingabematrix (durch Setzen des Parameters `overwrite_a=True`) einen Einfluss auf die Laufzeit hat. Die Ergebnisse zeigen, dass `lu_factor` mit `overwrite_a=False` die schnellste Variante darstellt und das Überschreiben der Matrix in der Regel zu einem kleinen Performanceverlust führt.

<div>


In [64]:
# Matrixgröße
n = 4096  # Die Matrix hat die Dimension 4096x4096

# Anzahl der Wiederholungen und der Ausführungen pro Wiederholung für die Zeitmessung
number = 1  # Jede Messung wird nur einmal ausgeführt
repetitions = 2  # Es wird 2 Wiederholungen der Zeitmessung durchgeführt, um eine genauere Schätzung zu erhalten

# Initialisiere den Random Number Generator (RNG) mit einem festen Seed für Reproduzierbarkeit
rng = np.random.default_rng(seed=123)

# Die lambda-Funktion definiert den zu wiederholenden Code, der jeweils eine der vier Varianten der LU-Zerlegung aufruft
repeat_lambda = lambda x: repeat(x, setup="A = rng.random((n, n))", repeat=repetitions, number=number, globals=globals())

# Wir führen die Zeitmessung für jede der vier Varianten der LU-Zerlegung durch
minimums = [
    min(repeat_lambda(x))  # Wir wählen die minimale Zeit aus den beiden Wiederholungen
    for x in [
        "lu_factor(A, overwrite_a=False)",  # LU-Zerlegung mit 'overwrite_a=False' und lu_factor
        "lu_factor(A, overwrite_a=True)",   # LU-Zerlegung mit 'overwrite_a=True' und lu_factor
        "lu(A, overwrite_a=False)",         # LU-Zerlegung mit 'overwrite_a=False' und lu
        "lu(A, overwrite_a=True)"           # LU-Zerlegung mit 'overwrite_a=True' und lu
    ]
]

# Die minimalen Laufzeiten der vier Varianten werden in der Liste 'minimums' gespeichert
minimums

[1.009529693000104, 1.048232730999871, 1.4839463319999595, 1.4356397540000216]

<div class="Toni_Markdown_Solution">

Die Laufzeitenanalyse zeigt, dass `lu_faktor` leicht schneller ist als `lu`. Je nach System können die Laufzeiten variieren.

</div>

**Aufgabe:** Untersuchen Sie den Speicherverbrauch der 4 Varianten mit memray.

Um den Einfluss der anderen Berechnungen zu minimieren, empfehlen wir hier, die Berechnungen in eine neue Datei zu schreiben und mit `!` in einer Code-Zelle memray's Command-Line Interface zu verwenden. 

Am besten tracen Sie nur die LU Funktion mit einem Context Manager!
Verwenden Sie jeweils folgende Dateinamen als `file_name` Parameter in `memray.Tracker`:

- lu_inplace.bin
- lu_notinplace.bin
- lu_factor_inplace.bin
- lu_factor_notinplace.bin

Beantworten Sie bitte die folgenden Fragen:
* Welche Version verwendet am meisten Speicher? 
* Bei welcher Version ist der Speicherverbrauch am konstantesten? 

_Points:_ 9

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

# Untersuchung des Speicherverbrauchs der LU-Zerlegung mit Memray

In dieser Aufgabe untersuchen wir den Speicherverbrauch von vier Varianten der LU-Zerlegung, die entweder die Funktion `lu` oder `lu_factor` verwenden, mit den Optionen `overwrite_a=True` und `overwrite_a=False`. Das Ziel ist es, den Speicherverbrauch für jede dieser Varianten zu messen und zu vergleichen, welche Version den meisten Speicher benötigt und bei welcher der Speicherverbrauch am konstantesten ist.

## Vorgehensweise

Wir verwenden das Profiling-Tool **Memray**, um den Speicherverbrauch zu messen. Memray verfolgt alle Speicherzuweisungen, die während der Programmausführung stattfinden. Dabei werden die Ergebnisse in einer binären Datei gespeichert. Nach der Ausführung analysieren wir diese Dateien, um den gesamten Speicherverbrauch zu ermitteln.

Die vier Varianten, die wir vergleichen, sind:

1. **lu_inplace**: Die Funktion `lu` wird mit der Option `overwrite_a=True` verwendet, was bedeutet, dass die Eingabematrix `A` in-place verändert wird.
2. **lu_notinplace**: Die Funktion `lu` wird mit der Option `overwrite_a=False` verwendet, was bedeutet, dass eine neue Matrix für die LU-Zerlegung erstellt wird.
3. **lu_factor_inplace**: Die Funktion `lu_factor` wird mit der Option `overwrite_a=True` verwendet.
4. **lu_factor_notinplace**: Die Funktion `lu_factor` wird mit der Option `overwrite_a=False` verwendet.

## Fragen zur Analyse

Die Aufgabe stellt zwei wichtige Fragen:

1. **Welche Version verwendet am meisten Speicher?**
2. **Bei welcher Version ist der Speicherverbrauch am konstantesten?**

## Ergebnisse

Die Messungen für den Speicherverbrauch der verschiedenen Varianten (in MB) ergeben die folgenden Werte:

- **Speichernutzung von lu_inplace**: 432 MB
- **Speichernutzung von lu_notinplace**: 560 MB
- **Speichernutzung von lu_factor_inplace**: 176 MB
- **Speichernutzung von lu_factor_notinplace**: 176 MB

### Antwort auf die Fragen:

1. **Welche Version verwendet am meisten Speicher?**
   - Die Variante `lu_notinplace` benötigt mit 560 MB den meisten Speicher.

2. **Bei welcher Version ist der Speicherverbrauch am konstantesten?**
   - Die Varianten `lu_factor_inplace` und `lu_factor_notinplace` verwenden mit jeweils 176 MB konstanten Speicher und sind deutlich speichereffizienter als die anderen Varianten.

## Fazit

Die Analyse zeigt, dass die `lu_factor`-Funktionen deutlich weniger Speicher benötigen als die `lu`-Funktionen. Das In-place-Verfahren (mit `overwrite_a=True`) führt zu einer geringeren Speicherbelegung als das Verfahren ohne In-place-Veränderung (`overwrite_a=False`), was auf die Vermeidung zusätzlicher Speicherzuweisungen hindeutet.

<div>

In [68]:
# Löschen der .bin-Dateien, wenn sie bereits existieren
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    if os.path.exists(file_name):  # Überprüfen, ob die Datei existiert
        os.remove(file_name)  # Löschen der Datei, wenn sie existiert

# Erstellen der .py-Dateien ohne Indentationsfehler
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    with open(file_name.replace(".bin", ".py"), "w") as file:        
        file.write(
f"""
import numpy as np
import memray
from scipy.linalg import lu_factor, lu

n = 4096
rng = np.random.default_rng(seed=123)
A = rng.random((n, n))

with memray.Tracker("{file_name}", native_traces=True):
    lu{"_factor" if "_factor" in file_name else ""}(A, overwrite_a={"_inplace" in file_name})
""") # Verwende Memray Tracker, um den Speicherverbrauch zu überwachen

<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">
memray muss installiert und importierbar sein, es wurde version 1.16.0 gemäß Dockerfile aus dem Kurs verwendet.
</div>

In [69]:
#alle .py files werden ausgefuehrt in python3 um .bin Dateien zu erstellen
#diese enthalten die Infos zum Speicherbedarf
#falls sie schon im Verzeichnis existieren, müssen sie zuvor gelöscht werden
# Alle .py-Dateien ausführen, um die .bin-Dateien zu erzeugen, die die Speichernutzungsdaten enthalten
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    os.system(f"python3 {file_name.replace('.bin', '.py')}")

In [70]:
#Speichernutzung wird aus allen .bin files gelesen und hier zusammengefasst
# Speichernutzung aus den .bin-Dateien lesen und ausgeben
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    reader = memray.FileReader(file_name)
    mem_use = 0

    # Alle Speicherzuweisungen durchgehen und die Gesamtgröße berechnen
    for allocation in reader.get_allocation_records():
        mem_use += allocation.size

    # Ausgabe des gesamten Speicherverbrauchs in MB
    print(f"Speichernutzung von {file_name[:-4]} : {round(mem_use / 2**20)}MB") 

Speichernutzung von lu_inplace : 432MB
Speichernutzung von lu_notinplace : 560MB
Speichernutzung von lu_factor_inplace : 176MB
Speichernutzung von lu_factor_notinplace : 176MB


<div style="color: darkblue; background-color: #e0f7ff; padding: 5px; border-radius: 5px;" title="Toni">

Die Funktionen mit Version `lu_factor` verwenden konstant nur `176 MB` an Speicher.

Funktionen mit Version `lu` verwenden deutlich mehr Speicher mit `432 MB` "inplace" und sogar `560 MB` "notinplace".

Der Speicherbedarf kann auf unterschiedlichen Systemen unterschiedlich ausfallen.

<div>


In [71]:
#Auch die command-line Funktion von memray kann verwendet werden
# äquivalent zu !memray stats lu_inplace.bin
os.system("memray stats lu_inplace.bin")

  Computing statistics... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00--:--
📏 Total allocations:
	18

📦 Total memory allocated:
	432.139MB

📊 Histogram of allocation size:
	min: 8.000B
	----------------------------------------
	< 42.000B  : 3 ▇▇▇▇▇▇▇▇▇▇▇▇▇
	< 222.000B : 0 
	< 1.148KB  : 2 ▇▇▇▇▇▇▇▇▇
	< 6.062KB  : 2 ▇▇▇▇▇▇▇▇▇
	< 32.000KB : 6 ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
	< 168.896KB: 0 
	< 891.443KB: 0 
	< 4.595MB  : 0 
	< 24.251MB : 1 ▇▇▇▇▇
	<=128.000MB: 4 ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
	----------------------------------------
	max: 128.000MB

📂 Allocator type distribution:
	 MALLOC: 15
	 CALLOC: 2
	 MMAP: 1

🥇 Top 5 largest allocating locations (by size):
	- lu:/opt/conda/lib/python3.11/site-packages/scipy/linalg/_decomp_lu.py:355 -> 160.016MB
	- lu:/opt/conda/lib/python3.11/site-packages/scipy/linalg/_decomp_lu.py:385 -> 128.000MB
	- lu:/opt/conda/lib/python3.11/site-packages/scipy/linalg/_decomp_lu.py:354 -> 128.000MB
	- asarray_chkfinite:/opt/conda/lib/python3.11/site-packages/numpy/lib/_

0

In [ ]:
#command-line Befehl zum Löschen aller erstellten files
os.system("rm -rf lu_*")